# Exchanging Solutions Between Two Nonmatching Spherical Surfaces

In [1]:
import numpy as np
from pydtk2 import *
from pymoab.core import Core
from pymoab.types import MBQUAD, MB_TYPE_DOUBLE, MB_TAG_DENSE, MBVERTEX
from mpi4py import MPI

In [2]:
choose_level = 4
assert choose_level >= 0 and choose_level <= 4 and isinstance(choose_level, int)

# the next line should be modified for maximum iterations
itmax = 200
# the next line indicates the Radial Basis Function Radius,
# if this value is too small, then the local problems may be singular,
# however, if it's too large, then it may take 'forever' to generate the operators
radius = 0.5

bo = WENLAND2


levels = []
mesh_root = 'meshes/untri/'
levels.append(
    (mesh_root+'CubSph1_0_0_0_1_41_55_63_GaussHillSlotCylnd.vtk',
     mesh_root+'sphere1.vtk')
)
levels.append(
    (mesh_root+'CubSph2_0_0_0_1_58_78_89_GaussHillSlotCylnd.vtk',
     mesh_root+'sphere2.vtk')
)
levels.append(
    (mesh_root+'CubSph3_0_0_0_1_82_110_126_GaussHillSlotCylnd.vtk',
     mesh_root+'sphere4.vtk')
)
levels.append(
    (mesh_root+'CubSph4_0_0_0_1_116_156_178_GaussHillSlotCylnd.vtk',
     mesh_root+'sphere8.vtk')
)

In [3]:
if not choose_level:
    coarse_file = 'meshes/CubSph2_0_0_0_41_55_63_GaussHillSlotCylnd.vtk'
    fine_file = 'meshes/CubSph2_0_0_0_73_81_101_GaussHillSlotCylnd.vtk'
else:
    coarse_file = levels[choose_level-1][1]
    fine_file = levels[choose_level-1][0]

In [4]:

# the next line should be modified for output file names
output_common = 'results/DTKresultsLevel%i' % choose_level
output_fine, output_coarse = output_common + '_fine', output_common + '_coarse'

In [5]:
# MOAB stuffs
mdb_fine, mdb_coarse = Core(), Core()

# load files, this will store point_data in tags
mdb_fine.load_file(fine_file)
mdb_coarse.load_file(coarse_file)

# the point data in vtk files are tagged with 'scalars'
field_fine = field_coarse = 'scalars'
if choose_level:
    verts = mdb_coarse.get_entities_by_type(0, MBVERTEX)
    ctag = mdb_coarse.tag_get_handle(
        field_coarse,
        size=1,
        tag_type=MB_TYPE_DOUBLE,
        storage_type=MB_TAG_DENSE,
        create_if_missing=True)
    dummy = np.zeros(verts.size(), dtype=np.float64)
    mdb_coarse.tag_set_data(ctag, verts, dummy)

In [6]:
# Now construct two PyDTK2 MOAB managers
fine_manager = DTK2MoabManager(mdb_fine)
coarse_manager = DTK2MoabManager(mdb_coarse)

In [ ]:
# now we push the tag strings into the managers
fine_manager.register_tag(field_fine)
coarse_manager.register_tag(field_coarse)

In [ ]:
# next step is to construct the DTK2 MOAB mappers
fine2coarse = DTK2Mapper(
    source=fine_manager,
    target=coarse_manager,
    r=radius,
    basis_order=bo
)
coarse2fine = DTK2Mapper(
    source=coarse_manager,
    target=fine_manager,
    r=radius,
    basis_order=bo
)

# now register coupled flags
fine2coarse.register_coupled_tags(field_fine, field_coarse)
coarse2fine.register_coupled_tags(field_coarse, field_fine)

In [ ]:
# the transfer loop
for it in range(itmax):
    # transfer from fine to coarse
    fine2coarse.apply(field_fine, field_coarse)
    # transfer from coarse to fine
    coarse2fine.apply(field_coarse, field_fine)
    
     # dump to file
    mdb_coarse.write_file(output_coarse + '%i.vtk' % it)
    mdb_fine.write_file(output_fine + '%i.vtk' % it)